Воробьев Петр ДЗ №1 по Графам

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

Я взял в качестве метрополитенов: Рим, Питер и Пхеньян

In [3]:
spb_con = pd.read_csv("spb_connections.csv")
spb_stations = pd.read_csv("spb_stations.csv")
rome_con = pd.read_csv("rome_connections.csv")
rome_stations = pd.read_csv("rome_stations.csv")

In [4]:
spb_con

,station_1,station_2
0,1,2
1,2,3
2,3,4
3,4,5
4,5,6
5,6,7
6,7,8
7,8,9
8,9,10
9,10,11


In [5]:
def make_sub_scheme(connections):
    g = nx.Graph()
    
    for i in range(len(connections)):
        _station_1, _station_2 = int(connections['station_1'][i]), int(connections['station_2'][i])
        g.add_edge(_station_1, _station_2)
        #g.add_edge(_station_2, _station_1)
    
    return g

In [6]:
t = make_sub_scheme(spb_con)
q = make_sub_scheme(rome_con)

In [7]:
from networkx import isomorphism

In [8]:
check = isomorphism.GraphMatcher(t, q)

In [9]:
check.is_isomorphic()

False

## Попробуем сравнять кол-во вершин(станций) и проверить на изоморфность еще раз

### удалять будем вершины степени 1

In [10]:
len(rome_stations), len(spb_stations)

(73, 61)

In [11]:
def sub_equalizer(graph, size_from, size_to):
    while size_from != size_to:
        idx = np.random.randint(low=1, high=size_from+1)
        try:
            if graph.degree(idx) == 1:
                graph.remove_node(idx)
            size_from -= 1
        except NameError or NetworkXError:
            continue
            
    return graph

In [12]:
new_rome = q

In [13]:
new_rome = sub_equalizer(new_rome, len(rome_stations), len(spb_stations))

In [14]:
check = isomorphism.GraphMatcher(t, new_rome)

In [15]:
check.is_isomorphic()

False

### Даже с удалением единичных вершин графы метро не изоморфны.

### Попробуем через BLISSS

In [46]:
import igraph as ig

In [55]:
spb_stations['id'].tolist()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61]

In [59]:
def make_ig(stations, connections):
    g = ig.Graph()
    edges = []
    idxs = stations['id'].tolist()
    n = len(idxs)
    
    g.add_vertices(n)
    g.vs["label"] = idxs
    
    for i in range(len(connections)):
        _station_1, _station_2 = int(connections['station_1'][i]), int(connections['station_2'][i])
        edges.append((_station_1, _station_2))
    g.add_edges
    
    return g

In [61]:
spb_ig = make_ig(spb_stations, spb_con)
rome_ig = make_ig(rome_stations, rome_con)

In [62]:
spb_ig.isomorphic_bliss(rome_ig)

False

### Питер и Рим не изоморфны также и по BLISS

### Попробуем найти изоморфный подграф в питерском метро по отношению к пхеньянскому.

In [38]:
pyon_stations = pd.read_csv("pyon_stations.csv")
pyon_connections = pd.read_csv("pyon_connections.csv")

In [39]:
pyon = make_sub_scheme(pyon_connections)

In [40]:
sub_iso = isomorphism.GraphMatcher(t, pyon)

In [41]:
sub_iso.subgraph_is_isomorphic()

True

In [42]:
_dict = sub_iso.mapping

In [43]:
for key in _dict:
    spb_node = spb_stations[spb_stations['id'] == key]
    pyon_node = pyon_stations[pyon_stations['id'] == _dict[key]]
    print(spb_node)
    print('~~~~~~ Isomorphed to ~~~~~~~')
    print(pyon_node)
    print('\n\n\n\n')

    id          name        line
10  11  Владимирская  red+orange
~~~~~~ Isomorphed to ~~~~~~~
   id     name   line
6   7  Samhung  green





   id              name line
4   5  Площадь Мужества  red
~~~~~~ Isomorphed to ~~~~~~~
   id      name   line
0   1  Kwangbok  green





   id    name line
5   6  Лесная  red
~~~~~~ Isomorphed to ~~~~~~~
   id        name   line
1   2  Potonggang  green





   id        name line
6   7  Выборгская  red
~~~~~~ Isomorphed to ~~~~~~~
   id         name   line
2   3  Hwsnggumbol  green





   id            name line
7   8  Площадь Ленина  red
~~~~~~ Isomorphed to ~~~~~~~
   id    name   line
3   4  Konsol  green





   id          name line
8   9  Чернышевская  red
~~~~~~ Isomorphed to ~~~~~~~
   id     name   line
4   5  Hyoksin  green





   id               name       line
9  10  Площадь Восстания  red+green
~~~~~~ Isomorphed to ~~~~~~~
   id     name       line
5   6  Chosung  green+red





    id           name   line
42  43  Ломоносовск

### Подграф найден и выведены изоморфные вершины